# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

# Подготовка данных

## Предобработка данных

Импортируем библиотеки, модуль Spark

In [ ]:
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, Imputer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder

RANDOM_SEED = 12345

Инициируем Spark-сессию

In [ ]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("Linear Rergession California") \
                    .getOrCreate()

### Загрузим датасет

In [ ]:
df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True)

Выведем на экран колонки и тип данных в датасете

In [ ]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Из задания нам известно, в колонках датасета содержатся следующие данные:  
        - **longitude** — широта;  
        - **latitude** — долгота;  
        - **housing_median_age** — медианный возраст жителей жилого массива;  
        - **total_rooms** — общее количество комнат в домах жилого массива;  
        - **total_bedrooms** — общее количество спален в домах жилого массива;  
        - **population** — количество человек, которые проживают в жилом массиве;  
        - **households** — количество домовладений в жилом массиве;  
        - **median_income** — медианный доход жителей жилого массива;  
        - **median_house_value** — медианная стоимость дома в жилом массиве;  
        - **ocean_proximity** — близость к океану.

Средстами Pandas методом describe посмотрим данные датафрейма

In [ ]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


### Поиск пропусков в данных

In [ ]:
for column in df.columns:
    check_col = F.col(column).isNull()
    print(column, df.filter(check_col == True).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


В колонке **total_bedrooms** обнаружено `207` пропуска. Заполним пропуски медианными значениями

In [ ]:
mean = df.select(F.mean('total_bedrooms')).collect()[0][0]
df = df.na.fill({'total_bedrooms': mean})

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Супер! Заполнил верно:)Еще можно вот так:


```python
    imputer = Imputer(inputCols=['total_bedrooms'],
                  outputCols=['total_bedrooms'])
    
    df = imputer.setStrategy('mean').fit(df).transform(df)
    
    
```

</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> ✔️ Посмотрел, спасибо за вариант ! </div>

Проверим, что не осталось пропусков

In [ ]:
for column in df.columns:
    check_col = F.col(column).isNull()
    print(column, df.filter(check_col == True).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


### Трансформируем категориальные признаки с помощью трансформера StringIndexer. Сделаем OHE-кодирование

In [ ]:
indexer = StringIndexer(inputCols=['ocean_proximity'],
                        outputCols=['ocean_proximity_idx'])
df = indexer.fit(df).transform(df)

encoder = OneHotEncoder(inputCols=['ocean_proximity_idx'],
                        outputCols=['ocean_proximity_ohe'])
df = encoder.fit(df).transform(df)

Посмотрим уникальные объекты колонки **ocean_proximity_idx**

In [ ]:
df.select('ocean_proximity_idx').distinct().show()

+-------------------+
|ocean_proximity_idx|
+-------------------+
|                0.0|
|                1.0|
|                4.0|
|                3.0|
|                2.0|
+-------------------+



Выведем на экран уникальные объекты колонки **ocean_proximity_ohe**

In [ ]:
df.select('ocean_proximity_ohe').distinct().show()

+-------------------+
|ocean_proximity_ohe|
+-------------------+
|      (4,[2],[1.0])|
|      (4,[0],[1.0])|
|          (4,[],[])|
|      (4,[3],[1.0])|
|      (4,[1],[1.0])|
+-------------------+



### Трансформация числовых признаков.  

Для числовых признаков тоже нужна трансформация — шкалирование значений — чтобы сильные выбросы не смещали предсказания модели.
Создадим список с числовыми признаками

In [ ]:
num_cols = ['longitude', 'latitude', 'housing_median_age',
        'total_rooms', 'population', 'households',
        'median_income']

target = 'median_house_value'

Собераем числовые признаки в один вектор

In [ ]:
numerical_assembler = VectorAssembler(inputCols=num_cols, outputCol="numerical_features")
df = numerical_assembler.transform(df)

Стандартизируем эти признаки

In [ ]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")
df = standardScaler.fit(df).transform(df)

Объединим числовые признаки с категориальными

In [ ]:
all_features = ['ocean_proximity_ohe','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features,
                                  outputCol="features")
df = final_assembler.transform(df)

df.select(all_features).show(3)

+-------------------+-------------------------+
|ocean_proximity_ohe|numerical_features_scaled|
+-------------------+-------------------------+
|      (4,[3],[1.0])|     [-61.007269596069...|
|      (4,[3],[1.0])|     [-61.002278409814...|
|      (4,[3],[1.0])|     [-61.012260782324...|
+-------------------+-------------------------+
only showing top 3 rows



### Разделение на выборки

Разделяем наш датасет на две части — выборку для обучения и выборку для тестирования качества модели с помощью метода randomSplit()

In [ ]:
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)

print(train_data.count(), test_data.count())

16431 4209


Вывод:  
    - ознакомились с датасетом  
    - произвели поиск пропусков в данных, заполнили их медианным значением по колонке  
    - трансформировали категориальные признаки, числовые признаки

## Обучение моделей

### Обучие модели используя все данные из файла

Укажем, какая колонка содержит вектор признаков для обучения, как называется колонка с целевой переменной.

In [ ]:
lr = LinearRegression(featuresCol = 'features', labelCol=target, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_data)

22/08/31 19:57:26 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/08/31 19:57:26 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Зададим функцию оценки модели

In [ ]:
def model_eval(model,test_data,target_col_name):
    predictions = model.transform(test_data)
    predictedLabes = predictions.select(target_col_name, "prediction")
    evaluator = RegressionEvaluator(labelCol=target_col_name, predictionCol="prediction")
    rmse = evaluator.evaluate(predictedLabes,{evaluator.metricName: "rmse"})
    r2 = evaluator.evaluate(predictedLabes, {evaluator.metricName: "r2"})
    mae = evaluator.evaluate(predictedLabes, {evaluator.metricName: "mae"})
    return rmse,r2,mae

Оценим модель и сохраним метрики в переменные для дальнейшего сравнения

In [ ]:
rmse,r2,mae = model_eval(lr_model,test_data,target)
print(rmse,r2,mae)

68058.44059022666 0.6553224019443017 49396.56401387112


### Обучение модели используя только числовые переменные, исключив категориальные

In [ ]:
lr2 = LinearRegression(featuresCol = 'numerical_features_scaled', labelCol=target, regParam=0.3, elasticNetParam=0.8)
lr2_model = lr2.fit(train_data)

 Оценим вторую модель и сохраним метрики

In [ ]:
rmse2,r22,mae2 = model_eval(lr2_model,test_data,target)
print(rmse2,r22,mae2)

69125.19371387232 0.6444327174483986 50653.7923946965


# Анализ результатов

Сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2

Построим сводную таблицу по метрикам

In [ ]:
scores = {"RMSE":[rmse,rmse2],
         "r2":[r2,r22],
         "MAE":[mae,mae2],
         }

scores = pd.DataFrame(scores,index=['lr1','lr2']).T

In [ ]:
scores

,lr1,lr2
RMSE,68058.440590,69125.193714
r2,0.655322,0.644433
MAE,49396.564014,50653.792395


### Вывод:  
  - обе модели показали удовлетворительный результат  
  - модель использующая категориальные данные оказалась немного лучше той, в которой использовали только числовые  

## Общий вывод:

В проекте нам необходимо было обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. Используя методы pySpark мы прочитали файл, ознакомись с его содержимым. Выполните предобработку данных: исследовали данные на наличие пропусков и заполнили их, выбрав медианное значение по колонке. Преобразовали колонку с категориальными значениями техникой One hot encoding.  


Построили две модели линейной регрессии на разных наборах данных:  
    - используя все данные из файла;  
    - используя только числовые переменные, исключив категориальные.  
    

Для построения модели использовали оценщик LinearRegression из библиотеки MLlib. Сравнили результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2. Анализ результатов показал что две модели почти одинаковые, расхождения результатов не значительные с использованием категориальных данных только, и с использованием только числовых.

In [ ]:
spark.stop()